In [2]:
import fitz  # PyMuPDF
import pandas as pd

def extract_pdf_attributes(pdf_path, page_number, output_csv_path):
    """
    指定したPDFファイルのブロック・ライン・スパン単位でデータ属性を取得し、CSV形式で出力。
    
    Args:
        pdf_path (str): PDFファイルのパス。
        page_number (int): ページ番号（1から始まる）。
        output_csv_path (str): 出力するCSVファイルのパス。
        
    Returns:
        pd.DataFrame: 取得したデータ属性を保持するデータフレーム。
    """
    try:
        # PDFを開く
        doc = fitz.open(pdf_path)
        
        # ページ番号が正しいか確認
        if page_number < 1 or page_number > len(doc):
            print(f"エラー: ページ番号は1から{len(doc)}の間で指定してください。")
            return None
        
        # 対象ページを取得（0インデックス）
        page = doc[page_number - 1]
        
        # ページ内のブロックを取得
        blocks = page.get_text("dict")["blocks"]
        
        # データ保持用リスト
        data = []
        
        for block_no, block in enumerate(blocks, start=1):
            if "lines" not in block:
                continue  # ブロックに行がない場合はスキップ
            
            for line_no, line in enumerate(block["lines"], start=1):
                if "spans" not in line:
                    continue  # 行にスパンがない場合はスキップ
                
                for span_no, span in enumerate(line["spans"], start=1):
                    # 各スパンの属性を取得
                    text = span.get("text", "")
                    bbox = line["bbox"]
                    font = span.get("font", "")
                    size = span.get("size", "")
                    color = span.get("color", "")
                    text_type = span.get("flags", "")
                    origin = span.get("origin", (None, None))  # originを取得
                    ascender = span.get("ascender", None)      # ascenderを取得
                    
                    # データをリストに追加
                    data.append({
                        "page_no": page_number,
                        "block_no": block_no,
                        "line_no": line_no,
                        "span_no": span_no,
                        "text": text,
                        "origin_x": origin[0],
                        "origin_y": origin[1],
                        "bbox_x0": bbox[0],
                        "bbox_y0": bbox[1],
                        "bbox_x1": bbox[2],
                        "bbox_y1": bbox[3],
                        "font": font,
                        "size": size,
                        "color": color,
                        "text_type": text_type,
                        "ascender": ascender,
                    })
        
        # PDFを閉じる
        doc.close()
        
        # pandasデータフレームに変換
        df = pd.DataFrame(data)
        
        # CSVとして出力
        df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")
        print(f"CSVファイルが出力されました: {output_csv_path}")
        
        return df
    except Exception as e:
        print(f"エラーが発生しました: {e}")
        return None


In [3]:
# 使用例
# 対象のPDFファイルを指定
#pdf_file = "ir/2025/34070_旭化成/24jp.pdf"  
base_dir = "ir/2025/"
#base_dir = "ir/2024/"

#pdf_dir =  "34070_旭化成"
#pdf_filename = "24jp.pdf"
pdf_dir =  "40040_レゾナック・ホールディングス"
pdf_filename = "pdf-sustainability-report-integratedreport-RESONAC24J_spread.pdf"
#pdf_dir =  "41880_三菱ケミカルグループ"
#pdf_filename = "23.pdf"

pdf_path = base_dir + pdf_dir + "/" + pdf_filename

page_num = 6  # 表示したいページ番号（1ページ目）
#extract_line_bbox_and_text(pdf_path, page_number)
#analyze_line_distances(pdf_file, page_num)
output_file_path = pdf_path[:len(pdf_path)-4] + "_attrib_page" + str(page_num) + ".csv"
# 属性を抽出し、CSVに出力
df = extract_pdf_attributes(pdf_path, page_num, output_file_path)

# pandasデータフレームを確認
if df is not None:
    print(df.head())



CSVファイルが出力されました: ir/2025/40040_レゾナック・ホールディングス/pdf-sustainability-report-integratedreport-RESONAC24J_spread_attrib_page6.csv
   page_no  block_no  line_no  span_no    text     origin_x    origin_y  \
0        6         1        1        1      10  1067.396851  660.833618   
1        6         2        1        1  データセクシ  1017.863770   23.243896   
2        6         2        2        1       ョ  1057.109619   23.243896   
3        6         2        3        1       ン  1062.807251   23.243896   
4        6         2        4        1   コーポレー   897.561646   23.243896   

       bbox_x0     bbox_y0      bbox_x1     bbox_y1               font  \
0  1067.396851  651.858154  1076.860962  663.472961       DIN2014-Bold   
1  1017.863770   14.761236  1058.427612   25.972237  UDShinGoPr6N-Bold   
2  1057.109619   14.761236  1064.196167   25.972237  UDShinGoPr6N-Bold   
3  1062.807251   14.761236  1069.893799   25.972237  UDShinGoPr6N-Bold   
4   897.561646   14.761236   931.391663   25.972237  UD